In [1]:
# Mount Google Drive if using Colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
    USING_COLAB = True
except ImportError:
    USING_COLAB = False

Mounted at /content/drive


In [2]:
!pip install nlpaug

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 7.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, MarianMTModel, MarianTokenizer

In [4]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from concurrent.futures import ThreadPoolExecutor

from nlpaug.util import Action

In [5]:
import spacy
from spacy.lang.es.stop_words import STOP_WORDS as STOP_WORDS_ES
from spacy.lang.en.stop_words import STOP_WORDS as STOP_WORDS_EN

In [6]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [7]:
import spacy
from nltk.corpus import stopwords
import string

In [ ]:
# # COLAB
# nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# # COLAB
# modelo en español de spaCy
# !python -m spacy download es_core_news_sm
# !python -m spacy download en_core_web_lg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 3.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


## Modelos

In [10]:
# model_ckpt = "papluca/xlm-roberta-base-language-detection"
# tokenizer_detected = AutoTokenizer.from_pretrained(model_ckpt)
# model_detected = AutoModelForSequenceClassification.from_pretrained(model_ckpt)

In [11]:
# model_name = "Helsinki-NLP/opus-mt-es-en"
# # model_name = "Helsinki-NLP/opus-mt-en-es"
# # model_name = "Helsinki-NLP/opus-mt-tc-big-en-es"

# tokenizer_tradu = MarianTokenizer.from_pretrained(model_name)
# model_tradu = MarianMTModel.from_pretrained(model_name)

## Carga de datos

In [12]:
pd.set_option('display.max_columns', None)

In [13]:
if USING_COLAB:
  # df_all = pd.read_csv("/content/drive/My Drive/GTSI/Codigos_ods/data/dataset.csv")
  # df_all = pd.read_csv("/content/drive/My Drive/GTSI/Codigos_ods/data/dataset_lang.csv")
  # df_all = pd.read_csv("/content/drive/My Drive/GTSI/Codigos_ods/data/dataset_english.csv")
  df_all = pd.read_csv("/content/drive/My Drive/GTSI/Codigos_ods/data/dataset_balansed.csv")

else:
  # df_all = pd.read_csv("./data/dataset.csv")
  # df_all = pd.read_csv("./data/dataset_lang.csv")
  # df_all = pd.read_csv("./data/dataset_english.csv")
  df_all = pd.read_csv("./data/dataset_balansed.csv")

In [14]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80144 entries, 0 to 80143
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   text      80144 non-null  object 
 1   value     80144 non-null  int64  
 2   lang      42140 non-null  object 
 3   gpu_used  38004 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.4+ MB


In [15]:
df_values_count = df_all["value"].value_counts().sort_values()

In [16]:
# df_values_count

In [17]:
df_values_count.mean()

np.float64(5009.0)

In [18]:
df_values_count.iloc[1]

np.int64(5009)

## Process

#### Detect Lang

In [19]:
# # batch_size = df_all.shape[0]//3
# batch_size = 100 #1448 # fueron los datos directos de la base de espol
# item = 2

In [20]:
# batch_size = 100

# result_list = []
# for i in range(15):
#   print("Index", i)
#   print("Desde:",batch_size * i, "hasta:", batch_size * (i + 1))
#   inputs = tokenizer_detected(df_all["text"].iloc[batch_size * i : batch_size * (i + 1)].to_list(), padding=True, truncation=True, return_tensors="pt")

#   with torch.no_grad():
#       logits = model_detected(**inputs).logits

#   preds = torch.softmax(logits, dim=-1)
#   id2lang = model_detected.config.id2label
#   vals, idxs = torch.max(preds, dim=1)
#   result_list.extend([(id2lang[k.item()], v.item()) for k, v in zip(idxs, vals)])
#   print("*"*50)

In [21]:
# data = {'list_col': result_list}
# df_result_lang = pd.DataFrame(data)

In [22]:
# len(result_list), df_result_lang.shape

In [23]:
# df_all["lang"] = df_result_lang['list_col'].str[0]

In [24]:
# # df_all[df_all["lang"].isna()]["lang"] = "en"
# df_all.loc[df_all["lang"].isna(), "lang"] = "en"
# # indices_na = df_all[df_all["lang"].isna()].index
# # df_all.iloc[indices_na, 2] = "en"

In [25]:
# df_all.info()

In [26]:
# # df_all["lang"].iloc[95:105]

In [27]:
# if USING_COLAB:
#   df_all.to_csv("/content/drive/My Drive/GTSI/Codigos_ods/data/dataset_lang.csv", index=False)
# else:
#   df_all.to_csv("./data/dataset_lang.csv", index=False)

In [28]:
# if USING_COLAB:
#   df_result_lang.to_csv("/content/drive/My Drive/GTSI/Codigos_ods/data/dataset_lang_estimate.csv", index=False)
# else:
#   df_result_lang.to_csv("./data/dataset_lang_estimate.csv", index=False)

#### Traduction

In [29]:
# df_all_cp = df_all.copy()

In [30]:
# df_all_cp[df_all_cp["lang"] == "es"].shape

In [31]:
# def get_translation(text):
#   translated = model_tradu.generate(**tokenizer_tradu(text, return_tensors="pt", padding=True))
#   return tokenizer_tradu.decode(translated[0], skip_special_tokens=True)

In [32]:
# # df_all_cp.loc[df_all_cp["lang"] == "es", "text"] = df_all_cp.loc[df_all_cp["lang"] == "es", "text"].apply(get_translation)

In [33]:
# # df_all_cp[df_all_cp["lang"] == "es", "text"]

In [34]:
# spanish_indices = df_all_cp[df_all_cp["lang"] == "es"].index
# total_spanish = len(spanish_indices)

# print(f"Total de textos en español a traducir: {total_spanish}")

In [35]:
# # Procesar en lotes de 100
# batch_size = 100
# for i in range(0, total_spanish, batch_size):
#     start_idx = i
#     end_idx = min(i + batch_size, total_spanish)
#     batch_indices = spanish_indices[start_idx:end_idx]
#     print("index", i)
#     print(f"Procesando lote {i // batch_size + 1}/{total_spanish // batch_size + 1}")

#     # Traducir el lote
#     batch_texts = df_all_cp.loc[batch_indices, "text"]
#     translated_batch = batch_texts.apply(get_translation)

#     # Actualizar el DataFrame
#     df_all_cp.loc[batch_indices, "text"] = translated_batch

#     print(f"Progreso: {end_idx}/{total_spanish} textos traducidos ({(end_idx/total_spanish)*100:.1f}%)")
#     print("*"*50)
# print("✅ Traducción completada!")

In [36]:
# if USING_COLAB:
#   df_all_cp.to_csv("/content/drive/My Drive/GTSI/Codigos_ods/data/dataset_english.csv", index=False)
# else:
#   df_all_cp.to_csv("./data/dataset_english.csv", index=False)

#### Balance

In [37]:
# df_all.info()

In [38]:
# # df_all.loc[df_all["lang"] == "es", "lang"]

In [39]:
# class_counts = df_all["value"].value_counts()
# target_count = class_counts.max()

In [40]:
# def simple_balance_with_bert(df, text_col='text', label_col='value'):
#     """Versión simplificada usando solo BERT"""

#     class_counts = df[label_col].value_counts()
#     target_count = class_counts.max()

#     # Solo BERT augmenter
#     aug = naw.ContextualWordEmbsAug(
#         model_path='bert-base-multilingual-uncased',
#         aug_p=0.1
#     )

#     augmented_data = []

#     for class_label, current_count in class_counts.items():
#         needed = target_count - current_count
#         print("Clase:", class_label, "con", current_count, "necesita", needed)

#         if needed <= 0:
#             continue

#         class_texts = df[df[label_col] == class_label][text_col].tolist()

#         for i in range(needed):
#             original_text = class_texts[i % len(class_texts)]
#             try:
#                 augmented_text = aug.augment(original_text)
#                 if isinstance(augmented_text, list):
#                     augmented_text = augmented_text[0]
#             except:
#                 augmented_text = original_text

#             augmented_data.append({
#                 text_col: augmented_text,
#                 label_col: class_label,
#                 "lang": "en"
#             })
#         print("*"*50)

#     return pd.concat([df, pd.DataFrame(augmented_data)], ignore_index=True)

In [41]:
# # balanced_df = simple_balance_with_bert(df_all)

In [ ]:
def balance_with_multi_gpu(df, text_col='text', label_col='value', gpu_ids=[0, 1]):
    """Balancea dataset usando múltiples GPUs"""

    # Verificar GPUs disponibles
    available_gpus = torch.cuda.device_count()
    print(f"GPUs disponibles: {available_gpus}")

    if available_gpus < len(gpu_ids):
        print(f"⚠️ Solo {available_gpus} GPUs disponibles, usando las primeras {available_gpus}")
        gpu_ids = gpu_ids[:available_gpus]

    # Crear augmenters para cada GPU
    augmenters = {}
    for gpu_id in gpu_ids:
        augmenters[gpu_id] = naw.ContextualWordEmbsAug(
            model_path='bert-base-multilingual-uncased',
            aug_p=0.1,
            device=f'cuda:{gpu_id}'
        )
        print(f"✅ Augmenter creado en GPU {gpu_id}")

    class_counts = df[label_col].value_counts()
    target_count = class_counts.max()

    def process_class_on_gpu(args):
        """Procesa una clase en una GPU específica"""
        class_label, needed_samples, class_texts, gpu_id = args

        augmenter = augmenters[gpu_id]
        augmented_data = []

        print(f"🚀 GPU {gpu_id}: Procesando clase {class_label} ({needed_samples} muestras)")

        batch_size = 16
        for i in range(0, needed_samples, batch_size):
            batch_end = min(i + batch_size, needed_samples)
            batch_texts = [class_texts[j % len(class_texts)] for j in range(i, batch_end)]
            print("batch_size", batch_size, "batch_end", batch_end, "needed_samples", needed_samples)

            try:
                augmented_batch = augmenter.augment(batch_texts)

                for aug_text in augmented_batch:
                    augmented_data.append({
                        text_col: aug_text,
                        label_col: class_label,
                        "lang": "en",
                        'gpu_used': gpu_id
                    })
            except Exception as e:
                print(f"Error en GPU {gpu_id}: {e}")
                # Fallback
                for text in batch_texts:
                    augmented_data.append({
                        text_col: text,
                        label_col: class_label,
                        "lang": "en",
                        'gpu_used': gpu_id
                    })

        print(f"✅ GPU {gpu_id}: Completada clase {class_label}")
        return augmented_data

    # Preparar trabajos para cada GPU
    jobs = []
    gpu_idx = 0

    for class_label, current_count in class_counts.items():
        needed = target_count - current_count
        if needed <= 0:
            continue

        class_texts = df[df[label_col] == class_label][text_col].tolist()
        gpu_id = gpu_ids[gpu_idx % len(gpu_ids)]

        jobs.append((class_label, needed, class_texts, gpu_id))
        gpu_idx += 1

    # Ejecutar en paralelo
    all_augmented_data = []
    with ThreadPoolExecutor(max_workers=len(gpu_ids)) as executor:
        results = executor.map(process_class_on_gpu, jobs)

        for result in results:
            all_augmented_data.extend(result)

    # Combinar resultados
    augmented_df = pd.DataFrame(all_augmented_data)
    balanced_df = pd.concat([df, augmented_df], ignore_index=True)

    print(f"\n🎉 Completado!")
    print(f"Total original: {len(df)}")
    print(f"Total final: {len(balanced_df)}")
    print(f"Muestras añadidas: {len(augmented_df)}")

    # Mostrar uso por GPU
    if len(augmented_df) > 0:
        gpu_usage = augmented_df['gpu_used'].value_counts()
        print("\nUso por GPU:")
        for gpu_id, count in gpu_usage.items():
            print(f"  GPU {gpu_id}: {count} muestras")

    return balanced_df

In [43]:
# Usar con 2 GPUs
# balanced_df = balance_with_multi_gpu(df_all, gpu_ids=[0])
# # balanced_df = balance_with_multi_gpu(df_all, gpu_ids=[0, 1])

In [44]:
# df_all.shape, balanced_df.shape

In [45]:
# balanced_df.info()

In [46]:
# if USING_COLAB:
#   balanced_df.to_csv("/content/drive/My Drive/GTSI/Codigos_ods/data/dataset_balansed.csv", index=False)
# else:
#   balanced_df.to_csv("./data/dataset_balansed.csv", index=False)

#### Lematizacion

###### ES

In [47]:
import re
# Lista de textos de ejemplo
texts = [
    "La inteligencia artificial está revolucionando la industria.",
    "Los avances en aprendizaje profundo permiten nuevas aplicaciones.",
    "La ética en inteligencia artificial es fundamental para su desarrollo.",
    "Muchos algoritmos de aprendizaje no supervisado son complejos.",
    "La visión por computadora es una rama de la inteligencia artificial."
]
texts = [re.sub(r'\W', ' ', text.lower()) for text in texts]

In [ ]:
# Cargar modelo en español
nlp = spacy.load("es_core_news_sm")

# Stopwords en español
stop_words = set(stopwords.words("spanish"))

# Función de preprocesamiento
def preprocess(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.lemma_ not in stop_words and not token.is_punct]
    return tokens

# Aplicar preprocesamiento
data_processed = [preprocess(text) for text in texts]

In [49]:
data_processed

[['inteligencia', 'artificial', 'revolucionando', 'industria'],
 ['avances', 'aprendizaje', 'profundo', 'permiten', 'nuevas', 'aplicaciones'],
 ['ética', 'inteligencia', 'artificial', 'fundamental', 'desarrollo'],
 ['algoritmos', 'aprendizaje', 'supervisado', 'complejos'],
 ['visión', 'computadora', 'rama', 'inteligencia', 'artificial']]

###### EN

In [ ]:
# Cargar modelo en ingles
# nlp = spacy.load("es_core_news_sm")
nlp = spacy.load("en_core_web_lg")

In [51]:
stopwords_es = set(stopwords.words('spanish'))
stopwords_en = set(stopwords.words('english'))
stop_words = stopwords_es | stopwords_en

In [ ]:
# Función de limpieza y lematización
def procesar_texto(texto):
    if pd.isna(texto):
        return ""
    # Unicode, minúsculas, quitar puntuación
    texto = texto.lower().translate(str.maketrans('', '', string.punctuation))
    # Procesar con spaCy
    doc = nlp(texto)
    # Lematizar y quitar stopwords
    tokens = [token.lemma_ for token in doc if token.lemma_ not in stop_words and not token.is_punct and not token.is_space]
    return " ".join(tokens)

# Unir columnas y procesar
def crear_corpus(row):
    return procesar_texto(str(row['text']))

In [53]:
df_all['CORPUS'] = df_all.apply(crear_corpus, axis=1)

In [54]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80144 entries, 0 to 80143
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   text      80144 non-null  object 
 1   value     80144 non-null  int64  
 2   lang      42140 non-null  object 
 3   gpu_used  38004 non-null  float64
 4   CORPUS    80144 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 3.1+ MB


In [55]:
if USING_COLAB:
  df_all.to_csv("/content/drive/My Drive/GTSI/Codigos_ods/data/dataset_balansed_lema.csv", index=False)
else:
  df_all.to_csv("./data/dataset_balansed_lema.csv", index=False)